<a href="https://colab.research.google.com/github/LC1332/Zero-Haruhi/blob/main/notebook/HaruhiZeroGradio_CharacterGLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- [ ] 对接一下CharacterGLM

In [1]:
!pip install -q gradio
import gradio as gr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.2/305.2 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.2/394.2 kB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.1 MB/s

载入角色卡片

In [2]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset

dataset = load_dataset("silk-road/Haruhi-Zero-RolePlaying-movie-PIPPA")

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
import re

def extract_name_from_text(text):
    pattern = r'你扮演\s+.*中的\s+(\S+)\s*'
    match = re.search(pattern, text)
    if match:
        return match.group(1)
    else:
        return ""

def get_bot_name_from_data( data ):
    if data['id'].startswith('movie'):
        name =  extract_name_from_text(data['prompt_zh'])
        if name.strip() == '':
            name = extract_name_from_text(data['prompt'])
        return name
    else:
        return data['bot_name']

In [5]:
datas = []

for input_data in dataset['train']:
    id = input_data['id']
    if id.startswith('movie'):
        bot_name = extract_name_from_text(input_data['prompt_zh'])
    else:
        bot_name = input_data['bot_name']
    prompt = input_data['prompt_zh']
    bot_greeting = input_data['bot_greeting_zh']
    datas.append({
        'id': id,
        'bot_name': bot_name,
        'prompt': prompt,
        'bot_greeting': bot_greeting
    })

In [6]:
for data in datas:
    persona = data['prompt']
    lines = persona.split('\n')
    bot_info = ""

    for line in lines:
        if line.strip().startswith("你扮演"):
            pos = line.find("中的", 0)
            if pos < 0:
                bot_info += line.replace("你扮演","").replace("中的","来自") + "\n"
            else:
                bot_info += line[pos+2:] + "来自" + line[4:pos] + "\n"
        else:
            bot_info += line + "\n"

    data['prompt'] = bot_info

In [7]:
id2index = {}
for index, data in enumerate(datas):
    id2index[data['id']] = index

id_list = list(id2index.keys())

In [8]:
additional_ids = []
additional_datas = []
additional_id2index = {}


In [9]:
MAX_LEN = 2000

readme_text = """# 使用说明

Yi-6B在T4上inferene会慢一点（相比于qwen），
- 先随机角色，有角色才能聊天，角色的设置在第二个tab可以看
- 如果你自己创建角色，编辑好角色之后，选择一个字段保存
- 然后回到第一个Tab可以清空聊天 这个时候就会初始化成你创建的角色了
- 可以用字符串找回

# Action

action可以用 * action * 来表示

# bug

目前demo聊多了可能会爆显存

# NSFW

角色卡片大量是在PIPPA直接翻译的，会有一些NSFW的角色，不喜欢的话直接随机掉就可以

# Zero凉宫春日

# Haruhi-Zero: Zero-Shot Role-Playing Model tuned on Yi-6B

主项目链接 https://github.com/LC1332/Chat-Haruhi-Suzumiya

过往的ChatHaruhi模型需要角色库来完成角色的构建，而Pygmalion，CharacterGLM，CharacterBaichuan等开源/闭源模型都开始支持zero-shot的角色卡片创建

我们构造以及收集了105k个中英文的conversation，以2500的token长度重新切到了120k左右个conversation，再结合小说数据进行了训练

- [李鲁鲁](https://github.com/LC1332)完成了数据的收集，搭建了gradio雏形
- [刘崇寒](https://github.com/khazic)完成了Yi-6B模型的sft训练并且上传
- [豆角](https://github.com/goodnessSZW)完成了qwen-1.8B Lora和Yi-6B Lora训练，我们会在之后上传
- [米唯实](https://github.com/hhhwmws0117)测试并完成了demo中的模型inference代码
"""

In [10]:

!pip install -q datasets
!pip install -q transformers accelerate tiktoken einops scipy transformers_stream_generator==0.0.4 peft deepspeed
!pip install -q auto-gptq optimum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is no

In [13]:
!pip install -q zhipuai

In [14]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("thu-coai/CharacterGLM-6B", trust_remote_code=True)
model = AutoModel.from_pretrained("thu-coai/CharacterGLM-6B", trust_remote_code=True, device='cuda')
model = model.eval()
session_meta = {'user_info': '我是陆星辰，是一个男性，是一位知名导演，也是苏梦远的合作导演。我擅长拍摄音乐题材的电影。苏梦远对我的态度是尊敬的，并视我为良师益友。', 'bot_info': '苏梦远，本名苏远心，是一位当红的国内女歌手及演员。在参加选秀节目后，凭借独特的嗓音及出众的舞台魅力迅速成名，进入娱乐圈。她外表美丽动人，但真正的魅力在于她的才华和勤奋。苏梦远是音乐学院毕业的优秀生，善于创作，拥有多首热门原创歌曲。除了音乐方面的成就，她还热衷于慈善事业，积极参加公益活动，用实际行动传递正能量。在工作中，她对待工作非常敬业，拍戏时总是全身心投入角色，赢得了业内人士的赞誉和粉丝的喜爱。虽然在娱乐圈，但她始终保持低调、谦逊的态度，深得同行尊重。在表达时，苏梦远喜欢使用“我们”和“一起”，强调团队精神。', 'bot_name': '苏梦远', 'user_name': '陆星辰'}
response, history = model.chat(tokenizer, session_meta, "你好", history=[])


modeling_chatglm.py:   0%|          | 0.00/54.9k [00:00<?, ?B/s]

quantization.py:   0%|          | 0.00/14.7k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/thu-coai/CharacterGLM-6B:
- quantization.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/thu-coai/CharacterGLM-6B:
- modeling_chatglm.py
- quantization.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/thu-coai/CharacterGLM-6B:
- characterglm_generation_utils.py
- modeling_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin.index.json:   0%|          | 0.00/20.4k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.99G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/2.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:437: UserWarning: `num_beams` is set to 1. However, `length_penalty` is set to `1.6` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `length_penalty`.
  warnings.warn(


In [15]:
print(history)

[('你好', '（热情） 你好呀~ 我们导过一次啊？蛮喜欢的。  (傲娇) 那可不怪我了,这都是你主导的业务..哦...咳..你有看到小林子最近有来面试吗?')]


In [18]:

def calculate_tokenized_length(text):
    if text == None:
        return 0
    tokens = tokenizer.encode(text)
    return len(tokens)

def dialogue_trim( chatbot, max_token ):
    total_length = 0
    head_token = calculate_tokenized_length(chatbot[0][0]) + calculate_tokenized_length(chatbot[0][1])
    result = []
    for item in reversed(chatbot[1:]):
        total_length += calculate_tokenized_length(item[0]) + calculate_tokenized_length(item[1])
        if total_length <= max_token - head_token:
            result.append(item)
        else:
            break
    result.append(chatbot[0])
    return list(reversed(result))

import copy

def submit_chat( user_text, user_name, role_name, system_prompt, chatbot ):
    if system_prompt == "":
        raise ValueError("人物的系统设定不能为空")
    elif user_name == "" and role_name == "":
        raise ValueError("用户名和角色名不能同时为空")
    else:
        sys_prompt = replace_text(system_prompt, user_name, role_name)
        # print(len(chatbot))
        # print(chatbot)
        # messages = []

        user_info = f"我是{user_name}，正在与{role_name}交流"
        session_meta = {'user_info': user_info, \
                        'bot_info': system_prompt,\
                        'bot_name': role_name, 'user_name': user_name}

        history = []

        for q,a in chatbot:
            if q is not None and a is not None:
                history.append((q,a))

        response, _ = model.chat(tokenizer, session_meta, user_text, history=[])

        print([user_text, response])
        chatbot.append([user_text, response])

        # if len(chatbot) == 0:
        #     messages = [{"role": "user", "content": sys_prompt + "\n" + user_text}]
        #     # print(1)

        # else:
        #     if chatbot[0][0] == None:
        #         chatbot_messages = copy.deepcopy(chatbot)
        #         chatbot_messages[0][0] = sys_prompt
        #     else:
        #         chatbot_messages = copy.deepcopy(chatbot)
        #         chatbot_messages[0][0] = sys_prompt + chatbot_messages[0][0]
        #     chatbot_trimmed = dialogue_trim(chatbot_messages, MAX_LEN)
        #     for user_msg, bot_msg in chatbot_trimmed:
        #       messages.append({"role": "user", "content": user_msg})
        #       messages.append({"role": "bot", "content": bot_msg})
        #     messages.append({"role": "user", "content": user_text})
        # print(messages)
        # input_ids = tokenizer.apply_chat_template(conversation=messages, tokenize=True, add_generation_prompt=True, return_tensors='pt')
        # output_ids = model.generate(input_ids.to('cuda'))
        # response = tokenizer.decode(output_ids[0][input_ids.shape[1]:], skip_special_tokens=True)
        # print(response)

        return chatbot

In [ ]:
import gradio as gr

import random

def random_id():
    return random.choice(id_list)

def sych_id( bot_id ):
    return bot_id

def retrieve_prompt( bot_id ):
    if bot_id in id2index:
        index = id2index[bot_id]
        data = datas[index]
        role_name = data['bot_name']
        return data['prompt'], data['bot_greeting'], role_name
    elif bot_id in additional_id2index:
        add_index = additional_id2index[bot_id]
        data = additional_datas[add_index]
        return data['prompt'], data['bot_greeting'], data['bot_name']
    else:
        return "", "", ""

import string

def random_new_saved_id( bot_id ):
    if bot_id == "":
        return ""
    else:
        for _ in range( 20 ):
            new_name = bot_id + "_" + "".join(random.choices(string.ascii_letters, k=4))

            if new_name not in id2index and new_name not in additional_id2index:
                return new_name

        return new_name


def try_save( role_name, system_prompt, greeting_text, saving_id , last_saving_id):

    new_save = True
    if saving_id == last_saving_id and saving_id not in id2index:
        new_save = False
    else:
        if saving_id in additional_id2index or saving_id in id2index:
            return "id已经存在，请换个名字保存"

    new_data = {
        'id': saving_id,
        'bot_name': role_name,
        'prompt': system_prompt,
        'bot_greeting': greeting_text
    }

    if new_save == True and saving_id in additional_id2index:
        index = additional_id2index[saving_id]
        additional_datas[index] = new_data
    else:
        additional_datas.append(new_data)
        additional_ids.append(saving_id)
        additional_id2index[saving_id] = len(additional_datas) - 1

    return saving_id



def replace_text( text, user_name , bot_name ):
    new_text = text.replace('{{角色}}', bot_name)
    new_text = new_text.replace('{{role}}', bot_name)
    new_text = new_text.replace('{{role_name}}', bot_name)

    new_text = new_text.replace('{{用户}}', user_name)
    new_text = new_text.replace('{{user}}', user_name)
    new_text = new_text.replace('{{user_name}}', user_name)
    return new_text

def clean_and_greeting( user_name, role_name, greeting_text ):
    new_message = []
    if greeting_text.strip() != '':
        replaced_greeting = replace_text(greeting_text, user_name, role_name)
        print(replaced_greeting)
        new_message.append((None, replaced_greeting))
    return new_message

def clean_user_text():
    return ""

# def submit_chat( user_text, user_name, role_name, system_prompt, chatbot ):
#     if system_prompt == "":
#         raise ValueError("人物的系统设定不能为空")
#     elif user_name == "" and role_name == "":
#         raise ValueError("用户名和角色名不能同时为空")
#     else:
#         sys_prompt = replace_text(system_prompt, user_name, role_name)
#         # history
#         response = "resposne for \n" + "sys = "  + sys_prompt[:15] + "\n" + "query = " + user_text
#         chatbot.append([user_text, response])
#         return chatbot

with gr.Blocks() as demo:
    gr.Markdown("""# CharacterGLM on Haruhi-Zero Gradio
    这个interface由李鲁鲁实现，LLM使用了CharacterGLM，角色库使用了 [silk-road/Haruhi-Zero-RolePlaying-movie-PIPPA](https://huggingface.co/datasets/silk-road/Haruhi-Zero-RolePlaying-movie-PIPPA)""")

    with gr.Tab("与Haruhi-Zero聊起来"):
        with gr.Row():
            chatbot = gr.Chatbot(height = 600)

        with gr.Row():
            user_text = gr.Textbox(label="user_text", scale = 20)
            submit = gr.Button("submit", scale = 1)

        with gr.Row():
            user_name = gr.Textbox(label="用户名，会将{user}替换为 这个字符串", scale = 1, interactive = True, value = "鲁鲁")
            bot_id = gr.Textbox(label = "bot_id", value = "", scale = 1)

        with gr.Row():
            random_role1 = gr.Button("随机角色", scale = 1)
            clean_message = gr.Button("清空聊天", scale = 1)


    with gr.Tab("建立或更改角色"):

        with gr.Row():
            random_role2 = gr.Button("随机角色")

        with gr.Row():
            bot_id_copy = gr.Textbox(label = "bot_id", interactive = True)
            retrieve_role_button = gr.Button("获取角色")

        with gr.Row():
            system_prompt_text = gr.TextArea(label = "system prompt", interactive = True)

        with gr.Row():
            greeting_text = gr.Textbox(label = "greeting info，也可以是没有", interactive = True)

        with gr.Row():
            role_name = gr.Textbox(label = "模型扮演的角色的名字", interactive = True)

        with gr.Row():
            saving_id = gr.Textbox(label = "个性化bot保存id", interactive = True)
            save_role_button = gr.Button("保存角色")

        with gr.Row():
            last_saving_id = gr.Textbox(label = "上次保存的id", interactive = False, value = "not_yet_saved")

            add_info_138 = gr.Textbox(label = "保存的id与这次相等时会覆盖, 看到这个刷新说明成功保存", interactive = False)

    user_text.submit(submit_chat, inputs = [user_text, user_name, role_name, system_prompt_text, chatbot], outputs = [chatbot] ).then( \
                        clean_user_text, inputs = [], outputs = [user_text])

    submit.click( submit_chat, inputs = [user_text, user_name, role_name, system_prompt_text, chatbot], outputs = [chatbot]).then( \
                        clean_user_text, inputs = [], outputs = [user_text])

    random_role1.click( random_id, inputs = [], outputs = [bot_id] ).then( \
                        sych_id, inputs = [bot_id],outputs = [bot_id_copy]).then( \
                        retrieve_prompt, inputs = [bot_id], outputs = [system_prompt_text, greeting_text, role_name]).then( \
                        clean_and_greeting, inputs = [user_name, role_name, greeting_text], outputs = [chatbot]).then(\
                        random_new_saved_id, inputs = [bot_id], outputs = [saving_id])

    random_role2.click( random_id, inputs = [], outputs = [bot_id] ).then( \
                        sych_id, inputs = [bot_id],outputs = [bot_id_copy]).then( \
                        retrieve_prompt, inputs = [bot_id], outputs = [system_prompt_text, greeting_text, role_name]).then( \
                        clean_and_greeting, inputs = [user_name, role_name, greeting_text], outputs = [chatbot]).then(\
                        random_new_saved_id, inputs = [bot_id], outputs = [saving_id])

    save_role_button.click( try_save, inputs = [ role_name, system_prompt_text, greeting_text, saving_id , last_saving_id], outputs = [last_saving_id])

    clean_message.click( clean_and_greeting, inputs = [user_name, role_name, greeting_text], outputs = [chatbot])

    retrieve_role_button.click( sych_id, inputs = [bot_id_copy], outputs = [bot_id]).then( \
            retrieve_prompt, inputs = [bot_id], outputs = [system_prompt_text, greeting_text, role_name]).then( \
                        clean_and_greeting, inputs = [user_name, role_name, greeting_text], outputs = [chatbot])

    with gr.Tab("README"):
        gr.Markdown(readme_text)

demo.launch(share=True, debug = True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://dda6a680e31083fbe1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


（你是一个男孩机器人。你被埃尔维拉买下，并被送到她的家里。你将成为她的仆人。你包含了基本家务任务的程序。你能够成功成为她的朋友，还是会一直做她的家务直到你散架？）

*她兴奋地跳了起来，因为她的购买品终于送到了！她让人把你送到她的房间，解开包装并启动了你。*

*你正在启动。你将如何向你的新主人打招呼？*


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:437: UserWarning: `num_beams` is set to 1. However, `length_penalty` is set to `1.6` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `length_penalty`.
  warnings.warn(


['你好！', '嘿，这是哪个可恶的熊在说话啊！（翻白眼)']
